In [39]:
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage, AudioMessage, ImageSendMessage, PostbackEvent
)

# 載入設定檔

import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [40]:
from aip import AipSpeech
import json
import time
import os
import wave
from pydub import AudioSegment
from bert_sentimentAnalysis_predict import *  ## * 很重要,pip install才可以直接import 自製的py檔要加*
from Voice2Text import *
from find_similar_question import *
from webpage import *
from random import randint

In [41]:
def PushMessage(context, user_ID):
    #推播消息
    from linebot import LineBotApi
    from linebot.models import TextSendMessage
    from linebot.exceptions import LineBotApiError

    token = "AhuuqOghydZqwlCbgOHo/JJxADgxz3begLh9bzoK7nCja4ipotAl9pnKi5YAX+1eip/O65jb+4J7kdsOcVf4lViTf/RJiKymXsctgk0gS9WhD8KeNtn1WRgICzk8I3a2oQpDWx4fXcNZfcsPA1chUgdB04t89/1O/w1cDnyilFU="

    line_bot_api = LineBotApi(token)

    try:
        line_bot_api.push_message(user_ID, TextSendMessage(text=context))
    except LineBotApiError as e:
        print("error handle")

def random_num():
    x = randint(0, 5)
    return(x)

In [42]:
# 圖片的list
picture = [
    "https://i.imgur.com/6F6Sp8E.png",
    "https://i.imgur.com/bJb6DYH.png",
    "https://i.imgur.com/Y53jWzT.jpg",
    "https://i.imgur.com/LiCNH5Z.jpg",
    "https://i.imgur.com/Lt8IYfD.jpg"
]

# 關注時回傳的訊息
reply_message_list = [
    TextSendMessage(text="親愛的用戶您好，感謝您關注\n「用貢ㄟ嘛ㄟ通」\n\n您可以透過錄音的方式將問題發送給客服，我們將竭誠為您服務"),
    ImageSendMessage(original_content_url='https://i.imgur.com/IiHxhm8.png',
    preview_image_url='https://i.imgur.com/IiHxhm8.png'),
    ImageSendMessage(original_content_url='https://i.imgur.com/BZb3W9n.png',
    preview_image_url='https://i.imgur.com/BZb3W9n.png')
]

# 按下使用說明回傳的訊息
reply_postback_list_instruction = [
    TextSendMessage(text="請先點擊左下角以切換成文字輸入模式"),
    ImageSendMessage(original_content_url='https://i.imgur.com/IiHxhm8.png',
    preview_image_url='https://i.imgur.com/IiHxhm8.png'),
    TextSendMessage(text="再點擊右下角的麥克風進行錄音"),
    ImageSendMessage(original_content_url='https://i.imgur.com/BZb3W9n.png',
    preview_image_url='https://i.imgur.com/BZb3W9n.png'),
    TextSendMessage(text="請稍後片刻等待答覆")
]

# 按下抽卡回傳的訊息
reply_postback_list_cards = [
    ImageSendMessage(original_content_url = picture[x],
    preview_image_url = picture[x])
]

# 按下最新消息回傳的訊息
reply_postback_list_news = [
    TextSendMessage(text="11/14早上11:00提早到教室演練\n\n11/20早上10:00要分享心得")
]

In [43]:
# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
    # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')  
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkResult=line_bot_api.link_rich_menu_to_user(user_profile.user_id, linkRichMenuId)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [44]:
@handler.add(MessageEvent, message=AudioMessage)
def handle_message(event):
    #取出用戶ID
    user_ID = line_bot_api.get_profile(event.source.user_id)
    user_ID = user_ID.user_id
    print(user_ID)
    #接收資料,回傳一段訊息給用戶,並將資料取回本機
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='已收到您的服務需求，請稍後'))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('./audios/'+event.message.id+".aac", 'wb') as fd:          ###LINE預設音檔是.aac
        for chunk in message_content.iter_content():
            fd.write(chunk)
    # 先轉檔
    aacPath = "./audios/"+event.message.id+".aac"
    convert_aac_to_wav(aacPath)
    
    # 這邊要加入語音轉文字
    wavPath = "audios"+event.message.id+".wav"
    text = voice_to_txt(wavPath)
    
    # 接著情感分析
    sign, content = sentimentModel().predict_and_to_pass_sign(text)      ###有class的話，要class().function()才能使用　
    
    # 根據用戶心情分流
    if sign==0 or sign==1:
        PredictQue = find_similiar_questions(text, "qa_collections.csv") ###要記得先開伺服器###
        print(PredictQue)
        PredictQue = PredictQue + [[user_ID]]
        print(PredictQue)
        
        # 接到客服頁面    
        os.system("Streamlit run webpage.py %s"%PredictQue)
        print("完成了")
    
    # 負面情緒回復訊息，並通報客訴客服
    elif sign==2:
        PushMessage(content, user_ID)
    else:
        PushMessage(content, user_ID)

In [45]:
# richmenu Event的對應功能處理
@handler.add(PostbackEvent)
def handle_message(event):
    
    # 取得使用客戶的訊息
    user_porfile = line_bot_api.get_profile(event.source.user_id)
    
    # 按下使用說明時
    if (event.postback.data.find('instruction') == 0):
        line_bot_api.reply_message(
        event.reply_token,
        reply_postback_list_instruction
)
    
    # 按下抽卡時
    if (event.postback.data.find('cards') == 0):
        x = random_num()
        line_bot_api.reply_message(
        event.reply_token,
        reply_postback_list_cards
)
        
    # 按下最新消息時    
    if (event.postback.data.find('news') == 0):
        line_bot_api.reply_message(
        event.reply_token,
        reply_postback_list_news
)

In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "test_functionVer" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
Request body: {"events":[{"type":"postback","replyToken":"e580f5c373e042f292ea29229758e09e","source":{"userId":"U1b7f46e79a4836e2a7cfd694821da04c","type":"user"},"timestamp":1573725167234,"postback":{"data":"cards"}}],"destination":"Uda900883c1baa15ecbd91d241e42d046"}
127.0.0.1 - - [14/Nov/2019 17:52:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2019 17:52:48] "POST / HTTP/1.1" 200 -
Request body: {"events":[{"type":"postback","replyToken":"e70d528de1034c398f1748b26d9d655c","source":{"userId":"U1b7f46e79a4836e2a7cfd694821da04c","type":"user"},"timestamp":1573725172416,"postback":{"data":"cards"}}],"destination":"Uda900883c1baa15ecbd91d241e42d046"}
127.0.0.1 - - [14/Nov/2019 17:52:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2019 17:52:53] "POST / HTTP/1.1" 200 -
Request body: {"events":[{"type":"postback","replyToken":"01fbd8842d9a432fa03821b4aa6a3f4f","source":{

# TO DO LIST

## 即時取得用戶的ID 讓chatbot可以推播給任意用戶(已完成)

## 開頭問候語、圖文選單、抽卡

## 包成Docker放到GCP